Let's connect to Chroma DB running in Docker. We'll need the `chromadb` package to interact with it.

In [1]:


import chromadb

client = chromadb.HttpClient(host="localhost", port=8000)

# Test connection
client.heartbeat()

ValueError: Could not connect to a Chroma server. Are you sure it is running?

The code above:
1. Installs the chromadb package
2. Imports chromadb and necessary settings
3. Creates an HTTP client connection to the Chroma instance running in Docker at localhost:8000
4. Tests the connection with a heartbeat check

Make sure your Docker container with Chroma is running before executing this code. The default port 8000 is used assuming standard Chroma Docker setup.


Let's connect to the 'mahasiswa' collection and perform some example queries to evaluate the data.

In [ ]:
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()

chroma_collection = client.get_collection("mahasiswa", embedding_function=embedding_function)

In [ ]:
# Get all documents

original_query = "Apa itu SPMB JATIM 2025?"
results = chroma_collection.query(
    query_texts=[original_query],
    n_results=5,
)

retrieved_documents = results["documents"][0]

for doc in retrieved_documents:
    print(doc)
    print('\n' + '-' * 80 + '\n')

# Leason 2 Pitfalls of retrival

In [ ]:
chroma_collection.count()

In [ ]:
import umap

embeddings = chroma_collection.get(include=["embeddings"])["embeddings"]
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)

In [ ]:
import numpy as np
from tqdm import tqdm


def project_embeddings(embeddings, umap_transform):
    umap_embeddings = np.empty((len(embeddings), 2))
    print(umap_embeddings.shape)
    for i, embedding in enumerate(tqdm(embeddings)):
        umap_embeddings[i] = umap_transform.transform([embedding])
    return umap_embeddings



In [ ]:
projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10)
plt.gca().set_aspect('equal', 'datalim')
plt.title('Projected Embeddings')
plt.axis('off')

Gambar tersebut merupakan visualisasi dari proyeksi embeddings menggunakan UMAP (Uniform Manifold Approximation and Projection). Visualisasi ini menunjukkan:

1. Representasi dimensi tinggi dari embeddings dokumen yang diproyeksikan ke dalam ruang 2D
2. Setiap titik pada plot mewakili satu dokumen/chunk teks dari dataset JUKNIS SPMB JATIM 2025
3. Titik-titik yang berdekatan menunjukkan dokumen yang memiliki konten/makna yang mirip
4. Titik-titik yang berjauhan menunjukkan dokumen dengan konten yang berbeda secara semantik

Visualisasi ini berguna untuk:
- Melihat sebaran/distribusi dokumen berdasarkan kemiripan kontennya
- Mengidentifikasi cluster/kelompok dokumen yang memiliki tema serupa
- Mendeteksi anomali atau outlier dalam dataset

In [ ]:
import matplotlib.pyplot as plt

# Tambahkan label untuk melihat konten dari beberapa titik sampel
plt.figure(figsize=(12, 8))
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10)

# Plot beberapa label sampel
for i in range(0, len(token_split_texts), len(token_split_texts) // 5):
    plt.annotate(token_split_texts[i][:50] + "...",
                 (projected_dataset_embeddings[i, 0], projected_dataset_embeddings[i, 1]),
                 fontsize=8)

plt.title('Projected Embeddings with Sample Labels')
plt.axis('off')
plt.show()


In [ ]:
query = "Apa saja syarat untuk mendaftar SPMB Jatim 2025?"
results = chroma_collection.query(
    query_texts=[query],
    n_results=5,
    include=['documents', 'embeddings']
)

retrieved_documents = results["documents"][0]

for doc in retrieved_documents:
    print(doc)
    print('\n' + '-' * 80 + '\n')

In [ ]:
query_embedding = embedding_function([query])[0]
retrieved_documents = results["embeddings"][0]

projected_query_embedding = project_embeddings([query_embedding], umap_transform)
projected_retrieved_embeddings = project_embeddings(retrieved_documents, umap_transform)

In [ ]:
# Plot the projected query and retrieved documents in the embedding space
plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10, color='gray')
plt.scatter(projected_query_embedding[:, 0], projected_query_embedding[:, 1], s=150, marker='X', color='r')
plt.scatter(projected_retrieved_embeddings[:, 0], projected_retrieved_embeddings[:, 1], s=100, facecolors='none',
            edgecolors='g')

plt.gca().set_aspect('equal', 'datalim')
plt.title(f'{query}')
plt.axis('off')

In [ ]:
# Query Expansion

In [ ]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

In [ ]:
def augment_query_generated(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": "Anda adalah seorang asisten yang sangat membantu dalam mencari sesuatu di dokumen petunjuk teknis. Berikan contoh jawaban untuk pertanyaan yang diberikan, yang mungkin ditemukan dalam dokumen seperti dokumen panduan, peraturan atau akademik."
        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
original_query = "Berikan overview singkat apa itu spmb dan tahapan tahapan apa saja yang perlu untuk dipersiapkan"
hypothetical_answer = augment_query_generated(original_query)

print("Hypotehnical Answer:", hypothetical_answer)
joint_query = f"{original_query} {hypothetical_answer}"


In [ ]:
results = chroma_collection.query(query_texts=joint_query, n_results=5, include=['documents', 'embeddings'])
retrieved_documents = results['documents'][0]

for doc in retrieved_documents:
    print(doc)
    print('\n' + '-' * 80 + '\n')

In [ ]:
retrieved_embeddings = results['embeddings'][0]
original_query_embedding = embedding_function([original_query])
augmented_query_embedding = embedding_function([joint_query])

projected_original_query_embedding = project_embeddings(original_query_embedding, umap_transform)
projected_augmented_query_embedding = project_embeddings(augmented_query_embedding, umap_transform)
projected_retrieved_embeddings = project_embeddings(retrieved_embeddings, umap_transform)

In [ ]:
import matplotlib.pyplot as plt

# Plot the projected query and retrieved documents in the embedding space
plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10, color='gray')
plt.scatter(projected_retrieved_embeddings[:, 0], projected_retrieved_embeddings[:, 1], s=100, facecolors='none',
            edgecolors='g')
plt.scatter(projected_original_query_embedding[:, 0], projected_original_query_embedding[:, 1], s=150, marker='X',
            color='r')
plt.scatter(projected_augmented_query_embedding[:, 0], projected_augmented_query_embedding[:, 1], s=150, marker='X',
            color='orange')

plt.gca().set_aspect('equal', 'datalim')
plt.title(f'{original_query}')
plt.axis('off')

In [ ]:
def augment_multiple_query(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": "Anda adalah asisten psisten yang sangat membantu dalam mencari sesuatu di dokumen peraturan petunjuk teknis. Anda mengajukan pertanyaan tentang dokumen di peraturan. "
                       "Sarankan hingga lima pertanyaan tambahan yang terkait untuk membantu mereka menemukan informasi yang mereka butuhkan, untuk pertanyaan yang diberikan. "
                       "Sarankan hanya pertanyaan singkat tanpa kalimat majemuk. Sarankan berbagai pertanyaan yang mencakup berbagai aspek topik."
                       "Pastikan pertanyaan-pertanyaan tersebut lengkap, dan berhubungan dengan pertanyaan awal."
                       "Keluarkan satu pertanyaan per baris. Jangan beri nomor pada pertanyaan."
        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    content = content.split("\n")
    return content

In [ ]:
original_query = "Apa saja syarat untuk mendaftar SPMB Jatim 2025?"
augmented_queries = augment_multiple_query(original_query)

for query in augmented_queries:
    print(query)


In [ ]:
queries = [original_query] + augmented_queries
results = chroma_collection.query(query_texts=queries, n_results=5, include=['documents', 'embeddings'])

retrieved_documents = results['documents']

# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

for i, documents in enumerate(retrieved_documents):
    print(f"Query: {queries[i]}")
    print('')
    print("Results:")
    for doc in documents:
        print(doc)
        print('')
    print('-' * 100)

In [ ]:
original_query_embedding = embedding_function([original_query])
augmented_query_embeddings = embedding_function(augmented_queries)

project_original_query = project_embeddings(original_query_embedding, umap_transform)
project_augmented_queries = project_embeddings(augmented_query_embeddings, umap_transform)


In [ ]:
result_embeddings = results['embeddings']
result_embeddings = [item for sublist in result_embeddings for item in sublist]
projected_result_embeddings = project_embeddings(result_embeddings, umap_transform)


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10, color='gray')
plt.scatter(project_augmented_queries[:, 0], project_augmented_queries[:, 1], s=150, marker='X', color='orange')
plt.scatter(projected_result_embeddings[:, 0], projected_result_embeddings[:, 1], s=100, facecolors='none',
            edgecolors='g')
plt.scatter(project_original_query[:, 0], project_original_query[:, 1], s=150, marker='X', color='r')

plt.gca().set_aspect('equal', 'datalim')
plt.title(f'{original_query}')
plt.axis('off')

In [ ]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": (
                "Kamu adalah asisten yang ahli dan membantu dalam mencari informasi dari buku petunjuk teknis. "
                "Pengguna akan mengajukan pertanyaan berdasarkan informasi yang diambil dari buku petunjuk teknis. "
                "Jawablah pertanyaan pengguna hanya berdasarkan informasi yang diberikan."
            )
        },
        {
            "role": "user",
            "content": f"Pertanyaan: {query} \n\nInformasi: {retrieved_documents}"
        }
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content


In [ ]:
text_answer = " ".join([doc for doc in results['documents'][0]])
print(text_answer)


Syarat untuk mendaftar SPMB Jatim 2025 dapat dilihat dalam lampiran vi dan vii keputusan terkait. Syarat tersebut mencakup daftar konsentrasi keahlian di SMA/SMK dan persyaratan khusus kesehatan di beberapa konsentrasi keahlian SMK. Lebih lanjut informasi mengenai persyaratan tersebut dapat ditemukan dalam peraturan dan ketentuan yang tertera dalam keputusan terkait SPMB Jatim 2025.
